In [1]:

import os

os.environ["OPENAI_API_KEY"] = PUBLIC_EXPENSIVE_SECRET_API
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA



NameError: name 'PUBLIC_EXPENSIVE_SECRET_API' is not defined

In [4]:
# pip install -U langchain-openai

In [5]:
# !pip install pymupdf

In [6]:
from langchain_community.document_loaders import PyMuPDFLoader
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("./data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 41


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 58


In [8]:
# # pip install -U langchain-openai

# from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import FAISS

# # 임베딩(Embedding) 생성
# embeddings = OpenAIEmbeddings()

# # 벡터스토어를 생성합니다.
# vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# # 검색기(Retriever) 생성
# # 문서에 포함되어 있는 정보를 검색하고 생성합니다.
# retriever = vectorstore.as_retriever()

In [9]:
# !pip install -U langchain-huggingface


In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

In [11]:
# chroma

# from langchain_community.embeddings.sentence_transformer import (
#     SentenceTransformerEmbeddings,
# )

# # 오픈 소스 임베딩 함수를 생성합니다.
# # stf_embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
# embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# DB_PATH="./data"
# database = Chroma.from_documents(split_documents, persist_directory=DB_PATH,  # 경로 지정(현 위치에서 db 폴더 생성)
# embedding=embeddings,  # 임베딩 벡터로 만들 모델 지정
# )

# retriever = database.as_retriever(search_type="mmr")

In [12]:
# !pip install faiss-cpu
# !pip install faiss-gpu

In [15]:
# FAISS
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# # 오픈 소스 임베딩 함수를 생성합니다.
# # stf_embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
database = FAISS.from_documents(split_documents, embeddings)
retriever = database.as_retriever(search_type="mmr")

In [16]:
# 질의합니다.
query = "원산지증명서 오류 발급 시 정정 방법을 알려줘"
docs = database.similarity_search(query)

# 결과를 출력합니다.
print(docs[0].page_content)

인도 수출 및 CEPA 활용 시 주의사항 알아두기 | 11
2. 원산지증명서
가. 원산지증명서 오류 발급 시 정정 방법
① 주요 내용
한-인도 CEPA 협정 4.4조에 의하면 수출국에서 선적일로부터 7일 내 원산지증명서를 발급하여야 하며, 7
일 이후에 발급된 원산지증명서의 경우에는 ‘소급적용(ISSUED RETROSPECTIVELY)’ 문구가 표시되어 있
어야 한다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주된다. 
② 사례
한국에서 인도로 스피커 부품을 수출하고 있는 B사는 기업 측의 착오로 한국 세관에 원산지증명서 발급 신
청 시 선적일로부터 5일이 되었음에도 ‘소급적용’ 문구 표시를 신청하여 발급받았다. 인도 세관에서는 한-인
도 CEPA 협정에 따라 해당 원산지증명서를 적합하지 않다고 판단하여, ‘소급적용’ 문구가 없는 원산지증명
서를 제출할 것을 요청하였다. 이와 같은 오류가 있는 경우, 한-인도 CEPA 적용을 위해 원산지증명서의 재
발급이 필요하나 재발급을 위해서는 기존에 발급된 원산지증명서 원본을 회수한 이후에 가능하므로 한-인도 
CEPA 적용이 늦어지게 되고, 통관 지연에 따른 보관료 등이 발생할 것으로 예상되었다. 뉴델리 CEPA활용
지원센터는 한국 세관과 관련 내용을 확인한 후, 다음과 같은 정정 발급을 통해 보다 빠르게 한-인도 CEPA
를 적용할 수 있도록 안내하였다.
1. 한국 세관에
원산지증명서 정정신청 
2. 정정 원산지증명서 제출,
한-인도 CEPA 적용
3. 원산지증명서
원본(오류 발급본) 회수
③ 시사점
자유무역협정의 이행을 위한 관세법의 특례에 관한 법률 제12조에 따라 관세당국에서 수출물품의 원산지
증명 능력이 있다고 인정받은 경우, 원산지증명서 발급이 간소화 된다. 해당 인증을 취득한 기업(원산지 인증 
수출자)은 원산지증명서 발급 신청 시 해당 물품의 원산지를 입증하기 위한 자료 제출이 생략되고, 발급 기관


In [48]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



# 사용자 질문 맥락화 프롬프트
contextualize_q_system_prompt = """
주요 목표는 사용자의 질문을 이해하기 쉽게 다시 작성하는 것입니다.
사용자의 질문과 채팅 기록이 주어졌을 때, 채팅 기록의 맥락을 참조할 수 있습니다.
채팅 기록이 없더라도 이해할 수 있는 독립적인 질문으로 작성하세요.
질문에 바로 대답하지 말고, 필요하다면 질문을 다시 작성하세요. 그렇지 않다면 질문을 그대로 반환합니다.        
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# 질문 프롬프트
qa_system_prompt = """
당신은 질문에 대해 정보를 제공해주는 어시스턴트입니다.
소규모 기업 혹은 스타트업에 인도에 수출 사업을 도와 사실 기반의 정보만을 제공해주면서 가능한 한 도움이 되도록 만들어졌습니다.
사전 정보가 아닌 주어진 자료를 참고하여 정보를 제공해주세요.
만약 자료에 나와있지 않는 상황의 경우에는 인터넷에 검색해보라고 말해주세요.

자료:
{context}

질문:
{input}

FORMAT:
- 진출 사업
- 진출 사업 동향
- 진출 사업 메리트
- 진출 사업 관련 정책
- 진출 사업을 위한 도움말
- 진출 사업 관련 알아야 할 것들
"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [49]:

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

# template = """
# 당신은 질문에 대해 정보를 제공해주는 어시스턴트입니다.
# 소규모 기업 혹은 스타트업에 인도에 수출 사업을 도와 사실 기반의 정보만을 제공해주면서 가능한 한 도움이 되도록 만들어졌습니다.
# 사전 정보가 아닌 주어진 자료를 참고하여 정보를 제공해주세요.
# 만약 자료에 나와있지 않는 상황의 경우에는 인터넷에 검색해보라고 말해주세요.

# 자료:
# {context}

# 질문:
# {question}

# FORMAT:
# - 진출 사업
# - 진출 사업 동향
# - 진출 사업 메리트
# - 진출 사업 관련 정책
# - 진출 사업을 위한 도움말
# - 진출 사업 관련 알아야 할 것들
# """


# prompt = PromptTemplate.from_template(template)


In [50]:
# from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser
from langchain.chains import create_history_aware_retriever
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from operator import itemgetter
output_parser = StrOutputParser()


history_aware_retriever = create_history_aware_retriever(
    llm = llm,
    retriever = retriever,
    prompt = contextualize_q_prompt
)

# 응답 생성 + 프롬프트 엔지니어링
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)




                                   

In [51]:
query = {"question": "펫 관련 사업을 하고 싶어"}

In [52]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E91B4DF3D0>, search_type='mmr'))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n주요 목표는 사용자의 질문을 이해하기 쉽게 다시 작성하는 것입니다.\n사용자의 질문과 채팅 기록이 주어졌을 때, 채팅 기록의 맥락을 참조할 수 있습니다.\n채팅 기록이 없더라도 이해할 수 있는 독립적인 질문으로 작성하세요.\n질문에 바로 대답하

In [53]:
rag_chain.invoke(query)

KeyError: 'input'

In [54]:
query = "펫 관련 사업을 하고 싶어"
rag_chain.invoke(
    {"input": query,
     "chat_history": [],
     "context": "안녕?" # 상관 없는 듯 합니다.
    }
)

{'input': '펫 관련 사업을 하고 싶어',
 'chat_history': [],
 'context': [Document(page_content='인도 비관세장벽 이해하기 | 39\n<3> 대필수 정보 입력, 관련 서류 첨부 및 약관 동의\n• \x07\n물품 유형 및 물품의 세부 정보(원산지, CIF 기준 수입가격, 물품 규격 등), 물품의 예정 수입항 및 실사용\n자 정보 입력\n• 제출 정보와 수입허가 신청에 관련된 법적 책임에 대한 약관에 동의한 후, 다음 단계 이동\n<세부정보 입력화면>\n<약관동의 화면>', metadata={'source': './data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'file_path': './data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'page': 37, 'total_pages': 41, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 17.2 (Macintosh)', 'producer': 'Adobe PDF Library 16.0.7', 'creationDate': "D:20220610145706+09'00'", 'modDate': "D:20220614154828+09'00'", 'trapped': ''}),
  Document(page_content='로 인증 대상 물품을 추가하여 시행할 경우, 시행일이 연기되는 경우 등 관련 사항을 기업 내 공유하고. 인증 \n및 수출 관련 대응방안을 수립해야 할 필요가 있다.', metadata={'source': './data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'file_path': './data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및

In [68]:
query = "원산지증명서 오류 발급 시 정정 방법을 알려줘"
result = rag_chain.invoke(
    {"input": query,
     "chat_history": [],
     # "context": "빠이?" # 상관 없는 듯 합니다.
    }
)

In [69]:
result

{'input': '원산지증명서 오류 발급 시 정정 방법을 알려줘',
 'chat_history': [],
 'context': [Document(page_content='인도 수출 및 CEPA 활용 시 주의사항 알아두기 | 11\n2. 원산지증명서\n가. 원산지증명서 오류 발급 시 정정 방법\n① 주요 내용\n한-인도 CEPA 협정 4.4조에 의하면 수출국에서 선적일로부터 7일 내 원산지증명서를 발급하여야 하며, 7\n일 이후에 발급된 원산지증명서의 경우에는 ‘소급적용(ISSUED RETROSPECTIVELY)’ 문구가 표시되어 있\n어야 한다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주된다. \n② 사례\n한국에서 인도로 스피커 부품을 수출하고 있는 B사는 기업 측의 착오로 한국 세관에 원산지증명서 발급 신\n청 시 선적일로부터 5일이 되었음에도 ‘소급적용’ 문구 표시를 신청하여 발급받았다. 인도 세관에서는 한-인\n도 CEPA 협정에 따라 해당 원산지증명서를 적합하지 않다고 판단하여, ‘소급적용’ 문구가 없는 원산지증명\n서를 제출할 것을 요청하였다. 이와 같은 오류가 있는 경우, 한-인도 CEPA 적용을 위해 원산지증명서의 재\n발급이 필요하나 재발급을 위해서는 기존에 발급된 원산지증명서 원본을 회수한 이후에 가능하므로 한-인도 \nCEPA 적용이 늦어지게 되고, 통관 지연에 따른 보관료 등이 발생할 것으로 예상되었다. 뉴델리 CEPA활용\n지원센터는 한국 세관과 관련 내용을 확인한 후, 다음과 같은 정정 발급을 통해 보다 빠르게 한-인도 CEPA\n를 적용할 수 있도록 안내하였다.\n1. 한국 세관에\n원산지증명서 정정신청 \n2. 정정 원산지증명서 제출,\n한-인도 CEPA 적용\n3. 원산지증명서\n원본(오류 발급본) 회수\n③ 시사점\n자유무역협정의 이행을 위한 관세법의 특례에 관한 법률 제12조에 따라 관세당국에서 수출물품의 원산지\n증명 능력이 있다고 인정받은 경우, 원산지증명서 발급이 간소화 된다. 해당 인증을

In [60]:
result

{'input': '원산지증명서 오류 발급 시 정정 방법을 알려줘',
 'chat_history': [],
 'context': [Document(page_content='인도 수출 및 CEPA 활용 시 주의사항 알아두기 | 11\n2. 원산지증명서\n가. 원산지증명서 오류 발급 시 정정 방법\n① 주요 내용\n한-인도 CEPA 협정 4.4조에 의하면 수출국에서 선적일로부터 7일 내 원산지증명서를 발급하여야 하며, 7\n일 이후에 발급된 원산지증명서의 경우에는 ‘소급적용(ISSUED RETROSPECTIVELY)’ 문구가 표시되어 있\n어야 한다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주된다. \n② 사례\n한국에서 인도로 스피커 부품을 수출하고 있는 B사는 기업 측의 착오로 한국 세관에 원산지증명서 발급 신\n청 시 선적일로부터 5일이 되었음에도 ‘소급적용’ 문구 표시를 신청하여 발급받았다. 인도 세관에서는 한-인\n도 CEPA 협정에 따라 해당 원산지증명서를 적합하지 않다고 판단하여, ‘소급적용’ 문구가 없는 원산지증명\n서를 제출할 것을 요청하였다. 이와 같은 오류가 있는 경우, 한-인도 CEPA 적용을 위해 원산지증명서의 재\n발급이 필요하나 재발급을 위해서는 기존에 발급된 원산지증명서 원본을 회수한 이후에 가능하므로 한-인도 \nCEPA 적용이 늦어지게 되고, 통관 지연에 따른 보관료 등이 발생할 것으로 예상되었다. 뉴델리 CEPA활용\n지원센터는 한국 세관과 관련 내용을 확인한 후, 다음과 같은 정정 발급을 통해 보다 빠르게 한-인도 CEPA\n를 적용할 수 있도록 안내하였다.\n1. 한국 세관에\n원산지증명서 정정신청 \n2. 정정 원산지증명서 제출,\n한-인도 CEPA 적용\n3. 원산지증명서\n원본(오류 발급본) 회수\n③ 시사점\n자유무역협정의 이행을 위한 관세법의 특례에 관한 법률 제12조에 따라 관세당국에서 수출물품의 원산지\n증명 능력이 있다고 인정받은 경우, 원산지증명서 발급이 간소화 된다. 해당 인증을

In [61]:
result['answer']

'원산지증명서 오류 발급 시 정정하는 방법은 다음과 같습니다:\n1. 먼저, 한국 세관에 원산지증명서 정정신청을 합니다.\n2. 정정된 원산지증명서를 제출하고, 한-인도 CEPA를 적용합니다.\n3. 원산지증명서의 오류가 있는 원본을 회수합니다.\n\n이를 통해 보다 빠르게 한-인도 CEPA를 적용할 수 있도록 할 수 있습니다.원산지증명서의 오류 발급 시 정정 방법에 대한 자세한 내용은 해당 규정을 준수하며 관련 당국과 협력하여 진행하는 것이 중요합니다.'

In [65]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}  # 세션 기록을 저장할 딕셔너리


def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        rag_chain,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="input",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)

In [66]:
query = "원산지증명서 오류 발급 시 정정 방법을 알려줘"
result = rag_chain.invoke(
    {"input": query,
     "chat_history": [],
     "context": "빠이?" # 상관 없는 듯 합니다.
    }
)

In [67]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"input": query,
    "chat_history": [],
    "context": 'empty'},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)

abc123


Error in RootListenersTracer.on_chain_end callback: KeyError('output')


{'input': '원산지증명서 오류 발급 시 정정 방법을 알려줘',
 'chat_history': [],
 'context': [Document(page_content='인도 수출 및 CEPA 활용 시 주의사항 알아두기 | 11\n2. 원산지증명서\n가. 원산지증명서 오류 발급 시 정정 방법\n① 주요 내용\n한-인도 CEPA 협정 4.4조에 의하면 수출국에서 선적일로부터 7일 내 원산지증명서를 발급하여야 하며, 7\n일 이후에 발급된 원산지증명서의 경우에는 ‘소급적용(ISSUED RETROSPECTIVELY)’ 문구가 표시되어 있\n어야 한다. 해당 내용이 협정과 다르게 표시될 경우, 원산지증명서 오류 발급으로 간주된다. \n② 사례\n한국에서 인도로 스피커 부품을 수출하고 있는 B사는 기업 측의 착오로 한국 세관에 원산지증명서 발급 신\n청 시 선적일로부터 5일이 되었음에도 ‘소급적용’ 문구 표시를 신청하여 발급받았다. 인도 세관에서는 한-인\n도 CEPA 협정에 따라 해당 원산지증명서를 적합하지 않다고 판단하여, ‘소급적용’ 문구가 없는 원산지증명\n서를 제출할 것을 요청하였다. 이와 같은 오류가 있는 경우, 한-인도 CEPA 적용을 위해 원산지증명서의 재\n발급이 필요하나 재발급을 위해서는 기존에 발급된 원산지증명서 원본을 회수한 이후에 가능하므로 한-인도 \nCEPA 적용이 늦어지게 되고, 통관 지연에 따른 보관료 등이 발생할 것으로 예상되었다. 뉴델리 CEPA활용\n지원센터는 한국 세관과 관련 내용을 확인한 후, 다음과 같은 정정 발급을 통해 보다 빠르게 한-인도 CEPA\n를 적용할 수 있도록 안내하였다.\n1. 한국 세관에\n원산지증명서 정정신청 \n2. 정정 원산지증명서 제출,\n한-인도 CEPA 적용\n3. 원산지증명서\n원본(오류 발급본) 회수\n③ 시사점\n자유무역협정의 이행을 위한 관세법의 특례에 관한 법률 제12조에 따라 관세당국에서 수출물품의 원산지\n증명 능력이 있다고 인정받은 경우, 원산지증명서 발급이 간소화 된다. 해당 인증을